In [2]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [12]:
LP1_seq = 'ccctaaaatgggcaaacattgcaagcagcaaacagcaaacacacagccctccctgcctgctgaccttggagctggggcagaggtcagagacctctctgggcccatgccacctccaacatccactcgaccccttggaatttcggtggagaggagcagaggttgtcctggcgtggtttaggtagtgtgagaggggaatgactcctttcggtaagtgcagtggaagctgtacactgcccaggcaaagcgtccgggcagcgtaggcgggcgactcagatcccagccagtggacttagcccctgtttgctcctccgataactggggtgaccttggttaatattcaccagcagcctcccccgttgcccctctggatccactgcttaaatacggacgaggacagggccctgtctcctcagcttcaggcaccaccactgacctgggacagtgaat'
print(len(LP1_seq))

LP1_seq = pad_seq(LP1_seq, 600, 'N', 'right')
LP1_seqs = [LP1_seq[:200], LP1_seq[200:400], LP1_seq[400:600]]
LP1_seqs_df = pd.DataFrame(LP1_seqs, columns=['seq'])
LP1_seqs_df.to_csv('../data/gene_therapy/LP1_seqs.csv')
LP1_seqs_df

447


,seq
0,ccctaaaatgggcaaacattgcaagcagcaaacagcaaacacacag...
1,cctttcggtaagtgcagtggaagctgtacactgcccaggcaaagcg...
2,cctgtctcctcagcttcaggcaccaccactgacctgggacagtgaa...


In [13]:
subprocess.run(
    ['python', 'predict_epi_features/0_predict_epi_feature.py', 
    '-i', 'data/gene_therapy/LP1_seqs.csv', 
    '-o', 'predict_epi_features/outputs/gene_therapy_LP1_seqs.h5',
    '-m', 'Sei', 
    '-d', 'cuda:0'],
    cwd='..'
)

predicting predict_epi_features/outputs/gene_therapy_LP1_seqs.h5


100%|██████████| 1/1 [00:00<00:00,  1.02it/s]


CompletedProcess(args=['python', 'predict_epi_features/0_predict_epi_feature.py', '-i', 'data/gene_therapy/LP1_seqs.csv', '-o', 'predict_epi_features/outputs/gene_therapy_LP1_seqs.h5', '-m', 'Sei', '-d', 'cuda:0'], returncode=0)

In [14]:
pred_array = load_h5('outputs/gene_therapy_LP1_seqs.h5')
print(pred_array.shape)

(3, 21907)


In [10]:
subprocess.run(
    ['python', 'train_scripts/test_0504.py', 
    '-s', 'saved/0612_Gosai_MPRA_MyResTransformer_seq_epi_3_cell_types/0611_230252/', 
    '-c', 'configs/config_0819_Gosai_MPRA_MyCNNTransformer_Zhang.yaml',
    '-o', 'Zhang_MPRA_pred.npy'],
    cwd='..'
)

use new config: configs/config_0819_Gosai_MPRA_MyCNNTransformer_Zhang.yaml


100%|██████████| 5/5 [00:01<00:00,  2.85it/s]


CompletedProcess(args=['python', 'train_scripts/test_0504.py', '-s', 'saved/0612_Gosai_MPRA_MyResTransformer_seq_epi_3_cell_types/0611_230252/', '-c', 'configs/config_0819_Gosai_MPRA_MyCNNTransformer_Zhang.yaml', '-o', 'Zhang_MPRA_pred.npy'], returncode=0)

In [16]:
cell_types = ['K562', 'HepG2', 'HEK293T', 'Jurkat']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']

In [17]:
MPRA_df = pd.read_csv("../data/Zhang_MPRA/Zhang_MPRA_Sei_feature_norm.csv")

In [18]:
y_pred = np.load(f'../saved/0612_Gosai_MPRA_MyResTransformer_seq_epi_3_cell_types/0611_230252/Zhang_MPRA_pred.npy')
cols = [f'{cell_type}_pred' for cell_type in cell_types]
MPRA_df[cols] = y_pred

In [19]:
pearson_df = pd.DataFrame()
for cell_type in cell_types:
    for cell_type_2 in cell_types:
        df = MPRA_df
        x = df[f'{cell_type}']
        y = df[f'{cell_type_2}_pred']
        r, p = pearson(x, y)
        pearson_df.loc[f'{cell_type}', f'{cell_type_2}_pred'] = r

print(pearson_df)

####### should be worst, but not

after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
         K562_pred  HepG2_pred  HEK293T_pred  Jurkat_pred
K562         0.720       0.645           NaN        0.647
HepG2        0.668       0.605           NaN        0.613
HEK293T      0.571       0.702           NaN        0.477
Jurkat       0.666       0.545           NaN        0.672
